In [ ]:
# --- 1. Route Optimization (Reinforcement Learning with TensorFlow) ---
import tensorflow as tf
import gymnasium as gym
import networkx as nx
import random
import numpy as np

# 1. Define the Environment (Road Network)
class DeliveryEnvironment(gym.Env):
    def __init__(self, graph, start_node, end_node):
        super(DeliveryEnvironment, self).__init__()
        self.graph = graph  # NetworkX graph
        self.start_node = start_node
        self.end_node = end_node
        self.current_node = start_node
        self.path = [start_node]  # Store the path taken
        self.action_space = gym.spaces.Discrete(len(list(graph.neighbors(start_node))))  # Number of neighbors
        self.observation_space = gym.spaces.Tuple((
            gym.spaces.Discrete(len(graph.nodes)),  # Current node
            gym.spaces.Discrete(len(graph.nodes)),  # End node
        ))

    def reset(self, *, seed=None, options=None):
        self.current_node = self.start_node
        self.path = [self.start_node]
        return self._get_obs(), {}

    def _get_obs(self):
        return (self.current_node, self.end_node)

    def step(self, action):
        next_node = list(self.graph.neighbors(self.current_node))[action]
        self.path.append(next_node)
        self.current_node = next_node

        if self.current_node == self.end_node:
            reward = -len(self.path)  # Negative path length (shorter is better)
            done = True
        else:
            reward = -1  # Small penalty for each step
            done = False

        observation = self._get_obs()
        info = {}

        return observation, reward, done, False, info

    def render(self):
        print(f"Current Node: {self.current_node}, Path: {self.path}")

# 2. Create a Sample Road Network (NetworkX)
# In a real scenario, you would load this from a database or API
graph = nx.Graph()
graph.add_edges_from([(0, 1), (0, 2), (1, 3), (2, 3), (3, 4)])
start_node = 0
end_node = 4

# 3. Build a Simple RL Model (TensorFlow) - Q-Network
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(2,)),  # Input: (current_node, end_node)
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(5, activation='linear')  # Output: Q-values for each action (neighbor)
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss_fn = tf.keras.losses.MeanSquaredError()

# 4. Training Loop
env = DeliveryEnvironment(graph, start_node, end_node)
episodes = 1000
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate

for episode in range(episodes):
    state, _ = env.reset()
    done = False

    while not done:
        # Epsilon-greedy action selection
        if random.random() < epsilon:
            action = env.action_space.sample()  # Explore
        else:
            state_tensor = tf.convert_to_tensor(np.array([state]))
            q_values = model(state_tensor)
            action = tf.argmax(q_values[0]).numpy()  # Exploit

        next_state, reward, done, _, _ = env.step(action)

        # Calculate target Q-value
        next_state_tensor = tf.convert_to_tensor(np.array([next_state]))
        next_q_values = model(next_state_tensor)
        target_q_value = reward + gamma * tf.reduce_max(next_q_values)

        # Calculate loss and update model
        with tf.GradientTape() as tape:
            state_tensor = tf.convert_to_tensor(np.array([state]))
            q_values = model(state_tensor)
            action_mask = tf.one_hot(action, depth=env.action_space.n)
            predicted_q_value = tf.reduce_sum(q_values * action_mask, axis=1)
            loss = loss_fn(target_q_value, predicted_q_value)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        state = next_state
    if (episode + 1) % 100 == 0:
        print(f"Episode {episode + 1}: Path: {env.path}, Reward: {reward}")

# --- 2. Delivery Time Prediction (Supervised Learning with XGBoost) ---
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# 1. Load and Prepare Data
# Assuming you have a CSV file with historical delivery data
# You can replace this with a dictionary if you're defining the data directly in the notebook
data = pd.read_csv('delivery_data.csv')  # Replace with your data file
# Feature Engineering
data['pickup_time'] = pd.to_datetime(data['pickup_time'])
data['dropoff_time'] = pd.to_datetime(data['dropoff_time'])
data['duration_seconds'] = (data['dropoff_time'] - data['pickup_time']).dt.total_seconds()
data['distance_km'] = data['distance_km'].astype(float)  # ensure distance is float
data['hour'] = data['pickup_time'].dt.hour
data['day_of_week'] = data['pickup_time'].dt.dayofweek
data['month'] = data['pickup_time'].dt.month

# Select Features and Target
features = ['pickup_location_lat', 'pickup_location_lng', 'dropoff_location_lat', 'dropoff_location_lng', 'distance_km', 'hour', 'day_of_week', 'month']  # make sure 'distance_km' is included
target = 'duration_seconds'
X = data[features]
y = data[target]

# 2. Split Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Train the XGBoost Model
model = xgb.XGBRegressor(
    objective='reg:squarederror',  # Regression objective
    n_estimators=100,  # Number of boosting rounds
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)
model.fit(X_train, y_train)

# 4. Make Predictions and Evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


# 5. Use the Model for Prediction
def predict_delivery_time(pickup_location_lat, pickup_location_lng, dropoff_location_lat, dropoff_location_lng, distance_km, hour, day_of_week, month):
    input_data = pd.DataFrame({
        'pickup_location_lat': [pickup_location_lat],
        'pickup_location_lng': [pickup_location_lng],
        'dropoff_location_lat': [dropoff_location_lat],
        'dropoff_location_lng': [dropoff_location_lng],
        'distance_km': [distance_km],
        'hour': [hour],
        'day_of_week': [day_of_week],
        'month': [month],
    })
    prediction = model.predict(input_data)[0]
    return prediction


# Example
predicted_time = predict_delivery_time(-1.28325, 36.82184, -1.30271, 36.78205, 10.5, 14, 1, 5)
print(f"Predicted Delivery Time: {predicted_time:.2f} seconds")